In [2]:
import os
import sys
import tarfile
import gif
import numpy as np
import igraph as ig
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_settings_v3 import *
from dismantling import get_index_list
from planar import create_proximity_graph, distance
from auxiliary import get_base_network_name

fig_dir = '../draft/figs'
param = 'param'

attack_dict = {
    'Ran': r'$\mathrm{Rnd}$', 'Deg': r'$\mathrm{ID}$', 'DegU': r'$\mathrm{RD}$',
    'Btw': r'$\mathrm{IB}$', 'BtwU': r'$\mathrm{RB}$', 
    'Eigenvector': r'$\mathrm{IE}$', 'EigenvectorU': r'$\mathrm{RE}$',
    'CI': r'$\mathrm{ICI}$', 'CIU': r'$\mathrm{RCI}$', 'CIU2': r'$\mathrm{RCI2}$',
}

In [ ]:
def get_position(net_type, size, net_dir=None):

    if net_type == 'Lattice':  
        L = size
        position = np.array([[i//L, i%L] for i in range(L*L)])
    else:
        N = size
        L = np.sqrt(N)
        ## Extract network file
        pos_tar_file_name = os.path.join(net_dir, 'position.tar.gz')
        tar = tarfile.open(pos_tar_file_name, 'r:gz')
        tar.extractall(net_dir)
        tar.close()

        pos_file_name = os.path.join(net_dir, 'position.txt')
        position = np.loadtxt(pos_file_name) * L
        os.remove(pos_file_name)
    
    return position

def get_max_pos(dir_name):
    
    tar_input_name = 'comp_data.tar.gz'
    full_tar_input_name = os.path.join(dir_name, tar_input_name)
    tar = tarfile.open(full_tar_input_name, 'r:gz')
    tar.extractall(dir_name)
    tar.close()

    full_file_name  = os.path.join(dir_name, 'comp_data.txt')
    aux = np.loadtxt(full_file_name)
    os.remove(full_file_name)

    Ngcc_values = aux[:,0][::-1] 
    delta_values = np.abs(np.diff(Ngcc_values))
    max_pos = np.argmax(delta_values)
    delta_max = delta_values[max_pos]
    
    return max_pos, delta_max

def load_data(net_type, size, param, attack, seed):
    
    dir_name = os.path.join('../networks', net_type)
    base_net_name, base_net_name_size = get_base_network_name(net_type, size, param)
    net_name = base_net_name_size + '_{:05d}'.format(seed)
    base_net_dir = os.path.join(dir_name, base_net_name, base_net_name_size)
    net_dir = os.path.join(base_net_dir, net_name)
    
    attack_dir_name = os.path.join(base_net_dir, net_name, attack)
    oi_file_name = os.path.join(attack_dir_name, 'oi_list.txt')
    index_list = np.loadtxt(oi_file_name, dtype='int')

    ## Extract network file
    tar_input_name = net_name + '.tar.gz'
    full_tar_input_name = os.path.join(net_dir, tar_input_name)
    tar = tarfile.open(full_tar_input_name, 'r:gz')
    tar.extractall(net_dir)
    tar.close()

    input_name = net_name + '.txt'
    full_input_name = os.path.join(net_dir, input_name)

    ## Read network file
    g = ig.Graph().Read_Edgelist(full_input_name, directed=False)

    ## Remove network file
    os.remove(full_input_name)
    
    g.vs['oi'] = range(g.vcount())
    g.vs['position'] = get_position(net_type, size, net_dir)
    g['attack_order'] = index_list
    
    max_pos, delta_max = get_max_pos(attack_dir_name)
    
    return g, max_pos, delta_max


L = 32
N = L*L
attack = 'BtwU'

size = {
    'Lattice': L,
    'DT': N
}
network = 'Lattice'
#network = 'DT'
seed = 0

#print(delta_max, delta_max*L, delta_max*L*L)

In [ ]:
@gif.frame
def plot(g, th, position):
    
    attack_order = g['attack_order']
    g.delete_vertices(attack_order[:th])

    gcc = g.components(mode='WEAK').giant()
    membership = np.array(g.components(mode='WEAK').membership)
    number, count = np.unique(membership, return_counts=True)
    number = number[np.argsort(count)[::-1]] 
    label_dict = dict((i, n) for i,n in enumerate(number))
    
    
    plt.figure(figsize=(10,10))

    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.xticks = range(L)
    plt.yticks = range(L)   
    plt.xlim(-1, L)
    plt.ylim(-1, L)
    
    if L <= 32:
        s = 40
    if L == 64:
        s = 15
    for oi in attack_order[:th]:
        plt.scatter(*position[oi], color='k', s=s)
        
    for oi in attack_order[th:]:
        idx = g.vs['oi'].index(oi)
        plt.scatter(*position[oi], color=colors[label_dict[membership[idx]]], s=s)
        
frames = []
th = 0
while True:
    
    g, max_pos, delta_max = load_data(network, size[network], param, attack, seed)    
    #position = np.array(g.vs['position'])
    position = g.vs['position']
    frame = plot(g, th, position)
    frames.append(frame)
    
    print(th, max_pos)
    if th == max_pos + 1:
        frames += [frame] * 10
        break
    th += 1
    
gif.save(frames, '{}_L{}_seed{:05}.gif'.format(network, L, seed), duration=100)
print('Finish')